# Preparations
Prepare the setup for scanning, as described in [scan_preparations.ipynb](scan_preparations.ipynb).

В трех отдельных сессиях терминала запустите, если еще не запущено, по одной строке из
```
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh all
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh info
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh err
```
В этих сессиях будут отображаться (1) все сообщения с информацией о сканировании, (2) сообщения об основных этапах сканирования, (3) ошибки.

## Start the initial camera warmup

In [ ]:
from pathlib import Path
import datetime
from time import sleep
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.scanning.scan_helper.scan_helper import ScanHelper

1. Initialize the ScanHelper and it will immediately start the camera warmup.
   Check in the logs that all 6 cameras are taking images.
   The phones click when they take a photo, so if you don't hear the clicking during the warmup
   this is an indicator that something goes wrong.
   Keep in mind that the phones pause the warmup for a couple of seconds for a cleanup after each 1k photos.
2. In a separate shell start the script for monitoring the temperature of the phones
```shell
python ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/phone_temp.py
```

In [ ]:
scans_root = '/mnt/data/sk3d/raw_scans'
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/logs/scanning/{today}_cameras.log'
started_warmup_at = datetime.datetime.now()

scan_helper = ScanHelper(scans_root, logfile, trajectories='spheres', thread='cameras')

## Set the white balance of the cameras
Setup the white balance once after starting the scan helper.
1. Turn off the room lights.
2. Plug the projector of the structured light scanner out.
3. Run the code below and follow the instructions.
4. Plug the projector of the structured light scanner back in.

<font color='red'>НЕ ЗАПУСКАЙТЕ эту клетку во время перемещения робота, вызванного из `scan_stl.ipynb`.</font>

In [ ]:
scan_helper.camera_trajectory_class().rest(.1)
scan_helper.set_white_balance()

## Scan

1. Смотрите этот шаг в `scan_stl.ipynb`.
2. Смотрите этот шаг в `scan_stl.ipynb`.
3. Задайте имя текущей сцены из таблицы.

In [ ]:
scene_name = 'large_coral_backpack'

4. Смотрите этот шаг в `scan_stl.ipynb`.
5. Смотрите этот шаг в `scan_stl.ipynb`.
6. Смотрите этот шаг в `scan_stl.ipynb`.
7. Смотрите этот шаг в `scan_stl.ipynb`.
8. Смотрите этот шаг в `scan_stl.ipynb`.
9. Смотрите этот шаг в `scan_stl.ipynb`.
10. Дождитесь прогрева камер.
* Если эта съемка --- первая после большого перерыва, то (1) ориентируйтесь на время ниже
  и (2) дождитесь выхода температуры телефонов на приблизительное плато.
* Если эта съемка после НЕбольшого перерыва, ориентируйтесь только на график температуры телефонов.

In [ ]:
print(f'Час с начала прогрева закончится в {(started_warmup_at + datetime.timedelta(hours=1)).strftime("%X")}.')

11. Проверьте правильность расположения ламп.
12. Загрузите настройки камер с диска.

In [ ]:
scan_helper.load_camera_settings(scene_name)

13. Снимите объект камерами. При перезапуске, сканирование продолжится с того положения на котором было прервано.

Если Вы хотите запустить сканирование сначала, перед сканированием запустите
```python
scan_helper.reset_camera_scanning_status(scene_name)
```

<font color='red'>НЕ ЗАПУСКАЙТЕ эту клетку во время перемещения робота, вызванного из `scan_stl.ipynb`.</font>

In [ ]:
scan_helper.scan_cameras(scene_name)

14. Смотрите этот шаг в `scan_stl.ipynb`.
15. Проверьте уровень батареи телефонов и свободное место на дисках.

Если свободного места на каком-либо из телефонов меньше 22Г или уровень батареи ниже 20%, то Вы что-то делаете не так.

In [ ]:
print(f'Left phone\tbattery {scan_helper.rig.phone_left.battery_level},\tfree disk space {scan_helper.rig.phone_left.disk_space}')
print(f'Right phone\tbattery {scan_helper.rig.phone_right.battery_level},\tfree disk space {scan_helper.rig.phone_right.disk_space}')

16. Скачайте данные с телефонов, как описано в видеоинструкции.

## Stop scanning
Для остановки камер в конце смены или перед перезапуском тетрадки запустите код ниже.

In [ ]:
scan_helper.stop()

Если камеры продолжают снимать, то, скорее всего, клетка зависла:
1. Перезапустите тетрадку,
2. Запустите прогрев камер,
3. запустите клетку выше снова.

Если камеры перестали снимать, то клетка должна завершится за 5-10 секунд.

Запустите клетку ниже и удостоверьтесь что экраны телефонов погасли.

In [ ]:
! for transport_id in $(adb devices -l | head -n3 | tail -n2 | cut -d' ' -f11 | cut -d':' -f2); do adb -t $transport_id shell 'input keyevent 224 && sleep 1 && input keyevent 26'; done

## Turn off the devices
Выполните шаги ниже если Ваша смена последняя.

**DO NOT put the plugs on metal surfaces, put them on plastic, wood, floor, or leave them in the air.**

1. Close the window with the plots of the phone temperatures and battery levels.
2. Close ScanCenter and `vm_server.bat` in the virtual machine, but leave the machine running.
3. If for some reason the robot is at some weird position, manually put it to some safe position.
4. Put the robot into the rest position with the code below.


<font color='red'>НЕ ЗАПУСКАЙТЕ эту клетку во время перемещения робота, вызванного из `scan_stl.ipynb`.</font>

In [ ]:
scan_helper.camera_trajectory_class().rest(.1)

5. <span style="background-color: #FFFF00">Перезапустите ядро тетрадки,</span> предварительно запустив `scan_helper.stop()` выше.
5. Stop and power off the robot.
6. Turn off the button on the main lights extender.
7. Unplug the main lights extender from the extender between the PC and the metal table.
8. Unplug the STL projector.

9. If the IR light emmiters of the Kinect are on, unplug it from the PC and plug it back.
10. If the IR projector of the RealSense is on, unplug it from the PC and plug it back.
11. Запустите клетку ниже и удостоверьтесь что экраны телефонов погасли.

In [ ]:
! for transport_id in $(adb devices -l | head -n3 | tail -n2 | cut -d' ' -f11 | cut -d':' -f2); do adb -t $transport_id shell 'input keyevent 224 && sleep 1 && input keyevent 26'; done